# Ajustarea fină a modelelor Open AI

Acest notebook se bazează pe ghidul actual oferit în documentația [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) de la Open AI.

Ajustarea fină îmbunătățește performanța modelelor de bază pentru aplicația ta prin reantrenarea acestora cu date suplimentare și context relevante pentru acel caz de utilizare sau scenariu specific. Reține că tehnici precum _few shot learning_ și _retrieval augmented generation_ îți permit să îmbunătățești promptul implicit cu date relevante pentru a crește calitatea. Totuși, aceste abordări sunt limitate de dimensiunea maximă a ferestrei de tokeni a modelului de bază vizat.

Prin ajustare fină, practic reantrenăm modelul cu datele necesare (ceea ce ne permite să folosim mult mai multe exemple decât ar încăpea în fereastra maximă de tokeni) și implementăm o versiune _personalizată_ a modelului, care nu mai are nevoie de exemple la momentul inferenței. Acest lucru nu doar că îmbunătățește eficiența designului promptului (avem mai multă flexibilitate în utilizarea ferestrei de tokeni pentru alte scopuri), dar poate reduce și costurile (scăzând numărul de tokeni pe care trebuie să-i trimitem modelului la inferență).

Ajustarea fină are 4 pași:
1. Pregătește datele de antrenament și încarcă-le.
1. Rulează jobul de antrenament pentru a obține un model ajustat fin.
1. Evaluează modelul ajustat fin și iterează pentru calitate.
1. Publică modelul ajustat fin pentru inferență când ești mulțumit de rezultate.

Reține că nu toate modelele de bază suportă ajustare fină – [verifică documentația OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) pentru cele mai noi informații. Poți ajusta fin și un model care a mai fost ajustat anterior. În acest tutorial, vom folosi `gpt-35-turbo` ca model de bază pentru ajustare fină.

---


### Pasul 1.1: Pregătește-ți setul de date

Hai să construim un chatbot care te ajută să înțelegi tabelul periodic al elementelor, răspunzând la întrebări despre un element cu un limerick. În _acest_ tutorial simplu, vom crea doar un set de date pentru a antrena modelul cu câteva exemple de răspunsuri care arată formatul așteptat al datelor. Într-un caz real, ar trebui să creezi un set de date cu mult mai multe exemple. Poți folosi și un set de date deschis (pentru domeniul tău de aplicație) dacă există unul, și să-l reformatezi pentru a-l folosi la fine-tuning.

Deoarece ne concentrăm pe `gpt-35-turbo` și vrem un răspuns într-o singură tură (chat completion), putem crea exemple folosind [acest format sugerat](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) care reflectă cerințele OpenAI pentru chat completion. Dacă ai nevoie de conținut conversațional pe mai multe ture, vei folosi [formatul de exemple multi-turn](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), care include un parametru `weight` pentru a semnala ce mesaje să fie folosite (sau nu) în procesul de fine-tuning.

Vom folosi formatul mai simplu, cu o singură tură, pentru tutorialul nostru de aici. Datele sunt în [format jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst), cu 1 înregistrare pe linie, fiecare reprezentată ca un obiect formatat JSON. Fragmentul de mai jos arată 2 înregistrări ca exemplu - vezi [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) pentru setul complet de exemple (10 exemple) pe care îl vom folosi pentru tutorialul nostru de fine-tuning. **Notă:** Fiecare înregistrare _trebuie_ să fie definită pe o singură linie (nu împărțită pe mai multe linii, cum este tipic într-un fișier JSON formatat)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Într-un caz real vei avea nevoie de un set de exemple mult mai mare pentru rezultate bune - va trebui să găsești un echilibru între calitatea răspunsurilor și timpul/costurile pentru fine-tuning. Noi folosim un set mic ca să putem termina fine-tuning-ul rapid și să ilustrăm procesul. Vezi [acest exemplu din OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) pentru un tutorial de fine-tuning mai complex.


### Pasul 1.2 Încarcă setul tău de date

Încarcă datele folosind Files API [așa cum este descris aici](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Ține cont că pentru a rula acest cod, trebuie să fi făcut următorii pași înainte:
 - Să fi instalat pachetul Python `openai` (asigură-te că folosești o versiune >=0.28.0 pentru cele mai noi funcționalități)
 - Să fi setat variabila de mediu `OPENAI_API_KEY` cu cheia ta OpenAI API
Pentru mai multe detalii, consultă [Ghidul de configurare](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) oferit pentru acest curs.

Acum, rulează codul pentru a crea un fișier de încărcat din fișierul tău local JSONL.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Pasul 2.1: Creează jobul de fine-tuning cu SDK-ul


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### Pasul 2.2: Verifică statusul jobului

Iată câteva lucruri pe care le poți face cu API-ul `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Listează ultimele n joburi de fine-tuning
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Afișează detalii despre un anumit job de fine-tuning
- `client.fine_tuning.jobs.cancel(<job_id>)` - Anulează un job de fine-tuning
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Listează până la n evenimente din job
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Primul pas al procesului este _validarea fișierului de antrenament_ pentru a te asigura că datele sunt în formatul corect.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Pasul 2.3: Urmărește evenimentele pentru a monitoriza progresul


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Pasul 2.4: Vizualizați statusul în OpenAI Dashboard


Poți vedea starea și accesând site-ul OpenAI și explorând secțiunea _Fine-tuning_ din platformă. Aici vei putea vedea starea jobului curent, dar și să urmărești istoricul execuțiilor anterioare. În această captură de ecran, poți observa că execuția anterioară a eșuat, iar a doua rulare a fost finalizată cu succes. Pentru context, acest lucru s-a întâmplat deoarece prima rulare a folosit un fișier JSON cu înregistrări formatate incorect – după ce problema a fost rezolvată, a doua rulare s-a încheiat cu succes și modelul a devenit disponibil pentru utilizare.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.ro.png)


Poți vedea și mesajele de stare și metricile derulând mai jos în panoul vizual, așa cum este prezentat:

| Mesaje | Metrice |
|:---|:---|
| ![Mesaje](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.ro.png) |  ![Metrice](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.ro.png)|


### Pasul 3.1: Recuperează ID-ul și testează modelul ajustat în cod


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### Pasul 3.2: Încarcă și testează modelul fine-tuned în Playground

Acum poți testa modelul fine-tuned în două moduri. În primul rând, poți accesa Playground și folosi meniul derulant Models pentru a selecta noul tău model fine-tuned din opțiunile afișate. Cealaltă opțiune este să folosești opțiunea "Playground" afișată în panoul Fine-tuning (vezi captura de ecran de mai sus), care deschide următoarea vizualizare _comparativă_ unde poți vedea versiunile modelului de bază și cel fine-tuned una lângă alta pentru o evaluare rapidă.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.ro.png)

Completează pur și simplu contextul de sistem folosit în datele tale de antrenament și adaugă întrebarea de test. Vei observa că ambele părți sunt actualizate cu același context și aceeași întrebare. Rulează comparația și vei vedea diferența dintre răspunsurile generate de cele două modele. _Observă cum modelul fine-tuned generează răspunsul în formatul pe care l-ai oferit în exemplele tale, în timp ce modelul de bază urmează doar promptul de sistem_.

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.ro.png)

Vei observa că această comparație oferă și numărul de tokeni pentru fiecare model, precum și timpul necesar pentru inferență. **Acest exemplu specific este unul simplificat, menit să arate procesul, dar nu reflectă un set de date sau un scenariu real**. Poți observa că ambele exemple afișează același număr de tokeni (contextul de sistem și promptul utilizatorului sunt identice), iar modelul fine-tuned are nevoie de mai mult timp pentru inferență (model personalizat).

În scenarii reale, nu vei folosi un exemplu atât de simplu, ci vei face fine-tuning pe date reale (de exemplu, un catalog de produse pentru suport clienți), unde calitatea răspunsului va fi mult mai evidentă. În _acel_ context, pentru a obține o calitate echivalentă a răspunsului cu modelul de bază, va fi nevoie de prompt engineering mai avansat, ceea ce va crește consumul de tokeni și, posibil, timpul de procesare pentru inferență. _Dacă vrei să testezi acest lucru, consultă exemplele de fine-tuning din OpenAI Cookbook pentru a începe._



---

**Declinarea responsabilității**:
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși depunem eforturi pentru acuratețe, vă rugăm să rețineți că traducerile automate pot conține erori sau inexactități. Documentul original, în limba sa nativă, trebuie considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist. Nu ne asumăm răspunderea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea în urma utilizării acestei traduceri.
